In [1]:
import torch
import torch.nn as nn

In [3]:
cfg = {
    'A':[64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B':[64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D':[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E':[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
    }

In [32]:
class VggNet(nn.Module):
    def __init__(self, features, in_channels=3, num_classes=1000, init_weights=True):
        super(VggNet, self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(7, 7))
        self.fc_layers = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )
        if init_weights:
            self._init_weights()

    def forward(self, x):
        # conv & pooling layers
        x = self.features(x)
        x = self.avgpool(x)
        # fc layers
        x = x.view(x.size(0), -1)
        x = fc_layers(x)
        return x

    def _init_weights(self):
        # class 내부에 정의 된 layer들을 가져옴
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # 뒤에 _를 붙이므로써 inplace=True의 효과를 냄
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 1)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.001)
                nn.init.constant_(m.bias, 0)

In [36]:
def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3

    for x in cfg:
        if type(x) == int:
            out_channels = x
            conv2d = nn.Conv2d(
                in_channels=in_channels, out_channels=out_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
            )
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(x), nn.ReLU()]
            else:
                layers += [conv2d, nn.ReLU()]
            in_channels = x
        else:
            layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

    return nn.Sequential(*layers)